In [8]:
## Two Mode Section
import numpy as np
import matplotlib.pyplot as plt
import time
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [9]:

# def Two_Mode_Optimizer(Seed_Parameters,Second_Best_Seed_Parameters,NumberOfSamples,Parameter_ChangeVector,Cost_Change,ImprovementIteration,TargetedSearchDecayRate,RandomSearchGrowthRate,TargetedMultiplicationFactor,RandomMultiplicationFactor,SignificantChangeValue,Maximal_Random_Search,Adaptive_Amplitude):
def Two_Mode_Optimizer(Seed_Parameters,Second_Best_Seed_Parameters,NumberOfSamples,Parameter_ChangeVector,Cost_Change,Random_Search_exponential_Growth_Factor,Targeted_Search_exponential_Growth_Factor,Targeted_Search_Decay_Rate,Random_Search_Growth_Rate,Targeted_Search_Growth_Rate,Random_Search_Period,Significant_Change_Value,Maximal_Random_Search,Adaptive_Amplitude,Directional_Pool_Ratio,Pool_Size,Directional_Pool):

#     Print_Two_Mode_Analytics = 0
#     if np.isnan(Cost_Change) == 1:
#         Print_Two_Mode_Analytics = 1
        
    Targeted_Search_Mechanism = 2


    Number_Of_Parameters_11 = len(Seed_Parameters)
    TargetedSearch = np.zeros((Number_Of_Parameters_11,NumberOfSamples))
    Random_Search_exponential_Growth_Factor1 = Random_Search_exponential_Growth_Factor
    

    ################################# Testing if the improvement since last iteration was big enough #############
    if np.sum(np.abs(Parameter_ChangeVector)) > Significant_Change_Value:
        Random_Search_exponential_Growth_Factor = 0
        Targeted_Search_exponential_Growth_Factor = Targeted_Search_exponential_Growth_Factor + 1

    if np.sum(np.abs(Parameter_ChangeVector)) < Significant_Change_Value:
        Targeted_Search_exponential_Growth_Factor = 0
        
        
    if Print_Two_Mode_Analytics == 1:
        print("Two_Mode_Analytics:")
        print("---------------------------------------------------------------------")
        print("The np.sum(np.abs(Parameter_ChangeVector)) is ",np.sum(np.abs(Parameter_ChangeVector)), )
        print("As Compared To the Significant_Change_Value of ", Significant_Change_Value)
    ###############################################################################################################        
        
        
        
        
    ################################# This the Random Search Section #############################################
    ## Applying a Genetic Algorithm
    # This is to broadcast them to a len(InitialParameters),NumberOfSamples size matrix, for element wise multiplication
    Seed_Parameters1 = np.zeros((Number_Of_Parameters_11,NumberOfSamples))
    Seed_Parameters1[:,:] = np.array([Seed_Parameters]).T
    Second_Best_Seed_Parameters1 = np.zeros((Number_Of_Parameters_11,NumberOfSamples))    
    Second_Best_Seed_Parameters1[:,:] = np.array([Second_Best_Seed_Parameters]).T    
    
    
    Random_Search_Scalar_Amplitude = Maximal_Random_Search*np.exp(Random_Search_Growth_Rate*(np.sin(np.mod(Random_Search_exponential_Growth_Factor*np.pi/(2*Random_Search_Period),np.pi/2) + 0.01)-1))
    Random_Samples = 2*(np.random.rand(Number_Of_Parameters_11,NumberOfSamples)-0.5)
    
    ## Take half of the parameter each of the best two individuals from last iteration + add a smart 
    ## RMS addaptive amplitude search around the children of the next generation (Mutation) + at the end 
    ## subtract the initial seed parameters so that we are only left with change vectors
    
    Genes = np.random.rand(Number_Of_Parameters_11,NumberOfSamples)


    RandomSearch = (Genes > 0.5)*Seed_Parameters1 + \
    (Genes <= 0.5)*Second_Best_Seed_Parameters1 + \
    Random_Samples*Random_Search_Scalar_Amplitude*Adaptive_Amplitude - Seed_Parameters1


    
    
    
    Number_Of_Directional_Samples = int(Directional_Pool_Ratio*NumberOfSamples)
    Directional_Samples = Random_Search_Scalar_Amplitude*np.matmul(Directional_Pool,np.random.rand(len(Directional_Pool[0,:]),Number_Of_Directional_Samples))
    RandomSearch[:,0:Number_Of_Directional_Samples] = Directional_Samples 
    
    
    
    if Print_Two_Mode_Analytics == 1:
        print("")
        print("Random_Search_Scalar_Amplitude = ",Random_Search_Scalar_Amplitude)
    ###############################################################################################################

        
    ################################# This the Targeted Search Section #############################################
    RandomNumbers = np.random.rand(NumberOfSamples,1)
    Random_Search_Scalar_Amplitude_1 = Maximal_Random_Search*np.exp(Random_Search_Growth_Rate*(np.sin(np.mod(Random_Search_exponential_Growth_Factor1*np.pi/(2*Random_Search_Period),np.pi/2) + 0.01)-1))

    
    if Targeted_Search_Mechanism == 1:
        TargetedSearch_Amplitude = RandomNumbers*np.exp(-Random_Search_exponential_Growth_Factor*Targeted_Search_Decay_Rate)     \
        *Targeted_Search_Growth_Rate*(-Cost_Change)

    if Targeted_Search_Mechanism == 2:
        TargetedSearch_Amplitude = Random_Search_Scalar_Amplitude_1*RandomNumbers*\
        np.exp(Targeted_Search_exponential_Growth_Factor*Targeted_Search_Growth_Rate\
               -Random_Search_exponential_Growth_Factor*Targeted_Search_Decay_Rate)

        
    if Print_Two_Mode_Analytics == 1:
        print("Mean Targeted_Search_Scalar_Amplitude =  = ",np.mean(TargetedSearch_Amplitude))
    
    if np.max(np.max(TargetedSearch_Amplitude)) > 1000: ## This is if the change vector is too big
#         print("Applied supression of change vector")
        
        if Targeted_Search_Mechanism == 1:
            TargetedSearch_Amplitude = RandomNumbers*np.exp(- Random_Search_exponential_Growth_Factor*Targeted_Search_Decay_Rate)     \
            *Targeted_Search_Growth_Rate*(10)

        
        if Targeted_Search_Mechanism == 2:
#             TargetedSearch_Amplitude = Random_Search_Scalar_Amplitude_1 *RandomNumbers*\
#             np.exp(30*Targeted_Search_Growth_Rate\
#                    -Random_Search_exponential_Growth_Factor1*Targeted_Search_Decay_Rate)
            TargetedSearch_Amplitude = TargetedSearch_Amplitude/10

        if Print_Two_Mode_Analytics == 1:
            print("np.mean(TargetedSearch_Amplitude) after suppression is = ",np.mean(TargetedSearch_Amplitude))
    
    if D <= 40:
        for sample in range (1,NumberOfSamples):
    #         TargetedSearch[:,sample] = TargetedSearch_Amplitude[sample]*Parameter_ChangeVector[:,0]
            TargetedSearch[:,sample] = TargetedSearch_Amplitude[sample]*Parameter_ChangeVector

    
    if D > 40:
        TargetedSearch = (np.matmul(np.matmul(np.diag(TargetedSearch_Amplitude[:,0]),np.ones((NumberOfSamples,Number_Of_Parameters_11))),np.diag(Parameter_ChangeVector))).T
    ###############################################################################################################
    
    
    ################################ This is the Final Readout and generating Suggestions ########################
    if Print_Two_Mode_Analytics == 1:   
        print("")
        print(" The Adaptive_Amplitude Vector is :",Adaptive_Amplitude[:,0])
        print("---------------------------------------------------------------------")
        print("")
    

    Suggested_Parameter_Samples_From_Two_Mode = np.array([Seed_Parameters]).T + TargetedSearch + RandomSearch
    Suggested_Parameter_Samples_From_Two_Mode[:,0] = Seed_Parameters
    
    Random_Search_exponential_Growth_Factor = Random_Search_exponential_Growth_Factor +1
    ##############################################################################################################

    return Suggested_Parameter_Samples_From_Two_Mode, Random_Search_exponential_Growth_Factor, Targeted_Search_exponential_Growth_Factor



In [ ]:
def Pinball_Optimizer(Initial_Parameters,NumberOfSamples,Search_Ratio,AmplitudeOfLinearSearch,AmplitudeOfRandomSearch,Linear_Search_Vector):


    ##Initalize infunction arrays
    Suggested_Parameter_Samples_From_Pinball = np.zeros((len(Initial_Parameters),NumberOfSamples))

    Number_Of_Linear_Samples = int(Search_Ratio*NumberOfSamples)

    Step_Size_Vector = np.linspace(-AmplitudeOfLinearSearch/2,AmplitudeOfLinearSearch/2,Number_Of_Linear_Samples)

    I = np.argmin(np.abs(Step_Size_Vector))
    Step_Size_Vector[I] = 0

    for k in range (0,NumberOfSamples):
        if k +1<=Number_Of_Linear_Samples:
                ## Generate Linear Search Samples around the inital parameter vector
            Suggested_Parameter_Samples_From_Pinball[:,k] = Initial_Parameters[:,0] + Linear_Search_Vector[:,0]*Step_Size_Vector[k]                     

        else:
                ## Generate Random Search Samples around the inital parameter vector  
            Suggested_Parameter_Samples_From_Pinball[:,k] = Initial_Parameters[:,0] + 2*AmplitudeOfRandomSearch*(np.random.rand(1,len(Initial_Parameters))-0.5)

    return Suggested_Parameter_Samples_From_Pinball

In [10]:
## Conductor Algorithm

def Symmetric_Section_Conductor(Current_Resource_Allocation,Changevector,Hyper_Parameters):
    ## The difference is that this function can allow for different masses and interactions between the algorithms
    
    
    Initial_Resource_Allocation = Hyper_Parameters[:,0]
    MassVector = Hyper_Parameters[:,1]
    Self_Interaction_Spring_Constants = Hyper_Parameters[:,2]
    InteractionMatrix = Hyper_Parameters[:,3:3+Number_Of_Algorithms]

    
    
    Norm = np.sqrt(np.matmul(Changevector,Changevector))
    if Norm != 0:
        Changevector = Changevector/Norm
    
    R0 = Initial_Resource_Allocation
    R = Current_Resource_Allocation
    K0 = Self_Interaction_Spring_Constants

    ## This updates the allocated resources
    H_Self = -(K0/MassVector)*(R-R0) ## This slowly restores the Resouce allocation to the original values
    H_Interaction = np.matmul(InteractionMatrix,(Changevector/MassVector).T) ## this is a resouce conserving interaction
    R = R + H_Self + H_Interaction.T
        

    RecomendedResourceAllocation1 = np.round(R)

    ## This makes sure that if the total resources aren't conserved then add or subtract from the best Member.
    I = np.argmax(RecomendedResourceAllocation1)
    RecomendedResourceAllocation1[I] = RecomendedResourceAllocation1[I] + (np.sum(Initial_Resource_Allocation) - np.sum(RecomendedResourceAllocation1))

    ## This convertes the array to an int array
    RecomendedResourceAllocation = np.arange(len(RecomendedResourceAllocation1))
    for i in range(len(RecomendedResourceAllocation1)):
        if np.isnan(RecomendedResourceAllocation1[i]):
#             print("RecomendedResourceAllocation1",RecomendedResourceAllocation1)
#             print("Changevector",Changevector)
            Changevector = np.zeros(4)
            RecomendedResourceAllocation1 = 15*np.ones(4)
        RecomendedResourceAllocation[i] = int(RecomendedResourceAllocation1[i])
    
    return RecomendedResourceAllocation





def Set_Hyperparameters_For_Symmetric_Conductor(Total_Resources,Number_Of_Algorithms,Mass,Self_Spring_Constants,Interaction_Spring_Constants):
    
#     Initial_Resource_Allocation = np.array([int(Total_Resources/Number_Of_Algorithms) ,int(Total_Resources/Number_Of_Algorithms) ,int(Total_Resources/Number_Of_Algorithms)])
    
    Initial_Resource_Allocation =int(Total_Resources/Number_Of_Algorithms)*np.ones(Number_Of_Algorithms)

    ## This fixes Rounding issues
    Initial_Resource_Allocation[0] = Initial_Resource_Allocation[0] + (Total_Resources-np.sum(Initial_Resource_Allocation))
    MassVector = Mass*np.ones(Number_Of_Algorithms)
    Self_Interaction_Spring_Constants = Self_Spring_Constants*np.ones(Number_Of_Algorithms)
    InteractionMatrix = Interaction_Spring_Constants*(-np.ones((Number_Of_Algorithms,Number_Of_Algorithms)) + Number_Of_Algorithms*np.diag(np.ones(Number_Of_Algorithms)))
    
    Hyperparameters_Array = np.zeros((Number_Of_Algorithms,3+Number_Of_Algorithms))
    Hyperparameters_Array[:,0] = Initial_Resource_Allocation
    Hyperparameters_Array[:,1] = MassVector
    Hyperparameters_Array[:,2] = Self_Interaction_Spring_Constants
    Hyperparameters_Array[:,3:3+Number_Of_Algorithms] = InteractionMatrix

    return Hyperparameters_Array



### Test of the conductor algorithm

Total_Resources = 60
Number_Of_Algorithms = 4
Mass =10
Self_Spring_Constants = 10
Interaction_Spring_Constants = 40
    
    
Hyperparameters_For_Symmetric_Conductor = Set_Hyperparameters_For_Symmetric_Conductor(Total_Resources,Number_Of_Algorithms,Mass,Self_Spring_Constants,Interaction_Spring_Constants)
Initial_Resource_Allocation = Hyperparameters_For_Symmetric_Conductor[:,0]
Current_Resource_Allocation = Hyperparameters_For_Symmetric_Conductor[:,0]



print("Initial Total_Resources",np.sum(Initial_Resource_Allocation))
for i in range (1,50):
    Changevector = np.array([-1, 0 ,0,0]) ## Negative is bad
    Recomended_Resource_Allocation = Symmetric_Section_Conductor(Current_Resource_Allocation,Changevector,Hyperparameters_For_Symmetric_Conductor) 
    Current_Resource_Allocation = Recomended_Resource_Allocation


Minimal_Allocation = np.min(Current_Resource_Allocation)
print("")
print("Minimum Resouces test")
print("------------------------------------------------------------------")
print("Minimal_Allocation is:",Minimal_Allocation)
print("(Maximal_Allocation is:",np.max(Current_Resource_Allocation),")")
print("Total_Resources After minumum resource allocation Test",np.sum(Current_Resource_Allocation))

for i in range (1,50):
    Changevector = np.array([1,0 ,0,0]) ## Positive is good
    Recomended_Resource_Allocation = Symmetric_Section_Conductor(Current_Resource_Allocation,Changevector,Hyperparameters_For_Symmetric_Conductor)
    Current_Resource_Allocation = Recomended_Resource_Allocation

Maximal_Allocation = np.max(Current_Resource_Allocation)
print("")
print("Maximum Resouces test")
print("------------------------------------------------------------------")
print("Maximal_Allocation is:",Maximal_Allocation)
print("(Minimal_Allocation is:",np.min(Current_Resource_Allocation),")")
print("Total_Resources After maximum resource allocation Test",np.sum(Current_Resource_Allocation))


Initial Total_Resources 60.0

Minimum Resouces test
------------------------------------------------------------------
Minimal_Allocation is: 3
(Maximal_Allocation is: 19 )
Total_Resources After minumum resource allocation Test 60

Maximum Resouces test
------------------------------------------------------------------
Maximal_Allocation is: 27
(Minimal_Allocation is: 11 )
Total_Resources After maximum resource allocation Test 60


In [29]:


### These are hyperparameters
# Search_Ratio,AmplitudeOfLinearSearch,AmplitudeOfRandomSearch
                
        
class Pinball_Suggestion_Tools(object):

    # The class "constructor" - It's actually an initializer 
    def __init__(self, Search_Ratio,Number_Of_Samples,Linear_Search_Vector,Best_Individual):
        self.Number_Of_Linear_Samples = int(Search_Ratio*Number_Of_Samples)
        self.Linear_Search_Vector    =    Linear_Search_Vector
        self.Best_Individual = Best_Individual
        self.Number_Of_Samples = Number_Of_Samples
        
def Create_New_Pinball_Suggestion_Tools_Object(Number_Of_Parameters,Search_Ratio = 0.7,Number_Of_Samples = 10): ## this is where the directional pool is initialized
#     N = Number_Of_Parameters
#     Number_Of_Samples = 10
#     Search_Ratio = 0.7
    Linear_Search_Vector = np.random.rand(Number_Of_Parameters) - 0.5
    Linear_Search_Vector = Linear_Search_Vector/np.sqrt(np.sum(Linear_Search_Vector**2))
    Best_Individual = np.random.rand(Number_Of_Parameters) - 0.5
    ## Generating and instance For The object the Pinball Suggestions tools object
    return  Pinball_Suggestion_Tools(Search_Ratio,Number_Of_Samples,Linear_Search_Vector,Best_Individual)







Test_Object_Functionality = 1
if Test_Object_Functionality == 1:
    ## Setting the inputs for the Two mode Suggestions tools object
    Number_Of_Parameters = 12
    Search_Ratio = 0.5
    Number_Of_Samples = 10
    a = Create_New_Pinball_Suggestion_Tools_Object(Number_Of_Parameters, Search_Ratio,Number_Of_Samples)
    print(a.Number_Of_Linear_Samples)
    print(a.Linear_Search_Vector)
    print(a.Best_Individual)

5
[ 0.15222119  0.48265059  0.42738488 -0.10128853 -0.2658909   0.06794057
 -0.42420935  0.26342591  0.02661909  0.27686681 -0.37500348 -0.09114726]
[-0.24244466 -0.11466386 -0.43025705  0.0952122   0.03824184  0.47298651
 -0.23152725  0.19561902 -0.35209843 -0.01299156 -0.04153837 -0.37235845]


In [27]:

# def Optimize_With_TSTM(InitialParameters,Cost1,NumberOfSamples,TargetedSearchDecayRate,RandomSearchGrowthRate,TargetedMultiplicationFactor ,RandomMultiplicationFactor,SignificantChangeValue,Maximal_Random_Search,print_Cost,Number_Of_Itterations,alpha,beta):
    
def Get_Pinball_Suggestions(fun,Hyperparameters,Suggestion_Tools,Print_Two_Mode_Analytics):
    

    

    Search_Ratio  = Hyperparameters[0]
    AmplitudeOfLinearSearch = Hyperparameters[1]
    AmplitudeOfRandomSearch = Hyperparameters[2]
    
    Best_Individual = Suggestion_Tools.Best_Individual
    Number_Of_Samples = Suggestion_Tools.Number_Of_Samples 
    Linear_Search_Vector = Suggestion_Tools.Linear_Search_Vector
    Number_Of_Linear_Samples = Suggestion_Tools.Number_Of_Linear_Samples
    
    
    Suggested_Parameter_Samples_Pinball_Mode = Pinball_Optimizer(Initial_Parameters,NumberOfSamples,Search_Ratio,AmplitudeOfLinearSearch,AmplitudeOfRandomSearch,Linear_Search_Vector)
    
    
    return Suggested_Parameter_Samples_Pinball_Mode





In [7]:

def fun(x):
    return np.sum((x-2)**2)


def Get_Suggestions_From_Pinball_Section(fun,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4,Hyperparameters_For_Symmetric_Conductor):

    ## This gets The Suggestions from The Different Sets Of hyperparameters Set 1

    Suggested_Parameter_Samples_From_Pinball_1,Suggestion_Tools_1  = Get_Pinball_Suggestions(fun,Hyperparameters_1,Suggestion_Tools_1 ,Print_Two_Mode_Analytics)   
    Suggested_Parameter_Samples_From_Pinball_2,Suggestion_Tools_2  = Get_Pinball_Suggestions(fun,Hyperparameters_2,Suggestion_Tools_2 ,Print_Two_Mode_Analytics)   
    Suggested_Parameter_Samples_From_Pinball_3,Suggestion_Tools_3  = Get_Pinball_Suggestions(fun,Hyperparameters_3,Suggestion_Tools_3 ,Print_Two_Mode_Analytics)   
    Suggested_Parameter_Samples_From_Pinball_4,Suggestion_Tools_4  = Get_Pinball_Suggestions(fun,Hyperparameters_4,Suggestion_Tools_4 ,Print_Two_Mode_Analytics) 


    ## This is to orginize the different indexes of the suggestions for later processing
    N1 = len(Suggested_Parameter_Samples_From_Two_Mode_1[0,:])
    N2 = len(Suggested_Parameter_Samples_From_Two_Mode_2[0,:])
    N3 = len(Suggested_Parameter_Samples_From_Two_Mode_3[0,:])
    N4 = len(Suggested_Parameter_Samples_From_Two_Mode_4[0,:])

    ## Save the indexes in the Suggestions array to know which samples came from which indvidual
    Indexes_Of_The_Suggestions_From_The_Different_Algorithms = np.array([0,N1,N1+N2,N1+N2+N3,N1+N2+N3+N4])
    ## Combine all the suggested samples into one array
    All_Suggestions_From_Pinball_Section = np.concatenate((Suggested_Parameter_Samples_From_Pinball_1,Suggested_Parameter_Samples_From_Pinball_2,Suggested_Parameter_Samples_From_Pinball_3,Suggested_Parameter_Samples_From_Pinball_4),1)


    return All_Suggestions_From_Pinball_Section,Indexes_Of_The_Suggestions_From_The_Different_Algorithms



In [8]:
   
def Get_Best_Cost_Change_From_Individuals(Cost_Change_From_Probe_Samples,Indexes_Of_The_Suggestions_From_The_Different_Algorithms1):

    Best_Cost_Change_From_Individuals  = np.zeros(4)
    Indexes = Indexes_Of_The_Suggestions_From_The_Different_Algorithms1

    for i in range (0,4):                                                                                                                       
        index_min = np.argmin(Cost_Change_From_Probe_Samples[Indexes[i]:Indexes[i+1]])
#         print("index_min For the individuals",index_min)
        Best_Cost_Change_From_Individuals[i] = Cost_Change_From_Probe_Samples[Indexes[i]:Indexes[i+1]][index_min]

    return  Best_Cost_Change_From_Individuals                                                                                

                                                                                                                                   
                                                                                                                                   
                                                                                                                        
         ## Update the Resouce Allocation
def  Update_Resource_Allocation(Cost_Change_From_Probe_Samples,Indexes_Of_The_Suggestions_From_The_Different_Algorithms1,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4,Hyperparameters_For_Symmetric_Conductor):
                                  
    # Normalize Cost
    if np.sum(np.abs(Cost_Change_From_Probe_Samples)) != 0:
#         Cost_Change_From_Probe_Samples = Cost_Change_From_Probe_Samples/(np.sqrt(np.matmul(Cost_Change_From_Probe_Samples,Cost_Change_From_Probe_Samples)))
                                                                            
        Cost_Change_From_Probe_Samples = Cost_Change_From_Probe_Samples/(np.sqrt(np.sum(Cost_Change_From_Probe_Samples**2)))
                                                                            
                 
            
            
                                                                        
    Best_Cost_Change_From_Individuals  = Get_Best_Cost_Change_From_Individuals(Cost_Change_From_Probe_Samples,Indexes_Of_The_Suggestions_From_The_Different_Algorithms1)                                                                                                                     
    Current_Resource_Allocation = np.array([Suggestion_Tools_1.Number_Of_Samples \
                                            ,Suggestion_Tools_2.Number_Of_Samples  \
                                            ,Suggestion_Tools_3.Number_Of_Samples  \
                                            ,Suggestion_Tools_4.Number_Of_Samples ])

    if print_Cost == 1:
        print("Current_Resource_Allocation",Current_Resource_Allocation)
    ## Important note - there is an extra minus so that negative best change is bad and positive best change is good 
    Recomended_Resource_Allocation = Symmetric_Section_Conductor(Current_Resource_Allocation,-Best_Cost_Change_From_Individuals,Hyperparameters_For_Symmetric_Conductor) 
    Current_Resource_Allocation = Recomended_Resource_Allocation

    Suggestion_Tools_1.Number_Of_Samples =      Current_Resource_Allocation[0]                                                                                                                     
    Suggestion_Tools_2.Number_Of_Samples =      Current_Resource_Allocation[1] 
    Suggestion_Tools_3.Number_Of_Samples =      Current_Resource_Allocation[2]                                                                                                                                  
    Suggestion_Tools_4.Number_Of_Samples =      Current_Resource_Allocation[3]    

    return Best_Cost_Change_From_Individuals



def Update_Best_Individuals_And_Linear_Search_Vector_From_This_Round(Best_Individual,Parameter_ChangeVector,Cost_Change,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4):                                                                                                                           


    Suggestion_Tools_1.Best_Individual = Best_Individual                                                                                                                    
    Suggestion_Tools_1.Second_Best_Individual = Second_Best_Individual
    Suggestion_Tools_1.Parameter_ChangeVector = Parameter_ChangeVector[:,0]                                                                                                                    
    Suggestion_Tools_1.Cost_Change = Cost_Change                                                                                                                     

    Suggestion_Tools_2.Best_Individual = Best_Individual                                                                                                                    
    Suggestion_Tools_2.Second_Best_Individual = Second_Best_Individual
    Suggestion_Tools_2.Parameter_ChangeVector = Parameter_ChangeVector [:,0]                                                                                                                   
    Suggestion_Tools_2.Cost_Change = Cost_Change                                                                                                                                        

    Suggestion_Tools_3.Best_Individual = Best_Individual                                                                                                                    
    Suggestion_Tools_3.Second_Best_Individual = Second_Best_Individual
    Suggestion_Tools_3.Parameter_ChangeVector = Parameter_ChangeVector[:,0]                                                                                                                    
    Suggestion_Tools_3.Cost_Change = Cost_Change                                                                                                                                        

    Suggestion_Tools_4.Best_Individual = Best_Individual                                                                                                                    
    Suggestion_Tools_4.Second_Best_Individual = Second_Best_Individual
    Suggestion_Tools_4.Parameter_ChangeVector = Parameter_ChangeVector[:,0]                                                                                                                    
    Suggestion_Tools_4.Cost_Change = Cost_Change  


    
    
    
    

def Update_Best_Individuals_And_Costs_From_This_Round(Best_Individual,Index_Of_Best_Individual,Indexes_Of_The_Suggestions_From_The_Different_Algorithms,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4):                                                                                                                           


     
    
    
    if Index_Of_Best_Individual > Indexes_Of_The_Suggestions_From_The_Different_Algorithms[0] and Indexes_Of_The_Suggestions_From_The_Different_Algorithms[0] < Indexes_Of_The_Suggestions_From_The_Different_Algorithms[0] + Suggestion_Tools_1.Number_Of_Linear_Samples:
        ## This means that the best sample was found in the directional search
        ## that means we should find a new one
        Linear_Search_Vector = (np.random.rand(len(Best_Individual )) - 0.5)
        Linear_Search_Vector = Linear_Search_Vector/np.sqrt(np.sum(Linear_Search_Vector**2))
        Suggestion_Tools_1.Linear_Search_Vector   = Linear_Search_Vector 
        
    else:
        Linear_Search_Vector = Best_Individual -  Suggestion_Tools_1.Best_Individual
        Linear_Search_Vector = Linear_Search_Vector/np.sqrt(np.sum(Linear_Search_Vector**2))
        Suggestion_Tools_1.Linear_Search_Vector   = Linear_Search_Vector      
    Suggestion_Tools_1.Best_Individual = Best_Individual                                                                                                       

    
    
    

#     Suggestion_Tools_2.Best_Individual = Best_Individual                                                                                                                    
#     Suggestion_Tools_2.Second_Best_Individual = Second_Best_Individual
#     Suggestion_Tools_2.Parameter_ChangeVector = Parameter_ChangeVector [:,0]                                                                                                                   
#     Suggestion_Tools_2.Cost_Change = Cost_Change                                                                                                                                        

#     Suggestion_Tools_3.Best_Individual = Best_Individual                                                                                                                    
#     Suggestion_Tools_3.Second_Best_Individual = Second_Best_Individual
#     Suggestion_Tools_3.Parameter_ChangeVector = Parameter_ChangeVector[:,0]                                                                                                                    
#     Suggestion_Tools_3.Cost_Change = Cost_Change                                                                                                                                        

#     Suggestion_Tools_4.Best_Individual = Best_Individual                                                                                                                    
#     Suggestion_Tools_4.Second_Best_Individual = Second_Best_Individual
#     Suggestion_Tools_4.Parameter_ChangeVector = Parameter_ChangeVector[:,0]                                                                                                                    
#     Suggestion_Tools_4.Cost_Change = Cost_Change  


    
    
    
    
    
    
def Optimize_With_TWO_Mode_Section(Best_Individual,Second_Best_Individual,PreviousCost,fun,current_iteration,Total_Number_Of_Iterations,Number_Of_Iterations_With_These_Hyperparameters,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4):

    
        
    Best_Cost_Change_From_Individuals1 = np.zeros((Number_Of_Algorithms,Number_Of_Iterations_With_These_Hyperparameters))
         
    if current_iteration == 0:
        Parameter_ChangeVector = np.zeros((len(Best_Individual),1))
        Cost_Change = 0
        Update_Best_Individuals_And_Costs_From_This_Round(Best_Individual,Second_Best_Individual,Parameter_ChangeVector,Cost_Change,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)                                                                                                                           

                                                                                                                          
    for i in range(0,Number_Of_Iterations_With_These_Hyperparameters):
        current_iteration= current_iteration + 1
        ## Readout ony if wanted                                                                                                                           
        if print_Cost == 1:
            print("____________________________________________________")
            print("Iteration",current_iteration,"\\",Total_Number_Of_Iterations)

        
                                                                                                                                
        ##  Get Sample Suggestions from the Section  
        All_Suggestions_From_Two_Mode_Section1,Indexes_Of_The_Suggestions_From_The_Different_Algorithms1 =  Get_Suggestions_From_Two_Mode_Section(fun,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4,Hyperparameters_For_Symmetric_Conductor)

        
        Cost = np.zeros(len(All_Suggestions_From_Two_Mode_Section1[1,:]))                                                                                                                         
        ## Sample the Function at the suggested sample points
        for j in range(0,len(All_Suggestions_From_Two_Mode_Section1[1,:])):
            Cost[j] =fun(All_Suggestions_From_Two_Mode_Section1[:,j])
            if np.isnan(Cost[j]):
                if j > 1:
                    Cost[j] = np.maximum(10**10,np.max(Cost[0:j-1]))
                else: 
                    Cost[j] = 10**10
                    
        ## Find the Optimzal Sample and calculate the change vectors                                                                                                                          
        index_min = np.argmin(Cost)
        CurrentCost = Cost[index_min ]
        Cost_Change = CurrentCost - PreviousCost
        Parameter_ChangeVector = np.array([All_Suggestions_From_Two_Mode_Section1[:,index_min] - Best_Individual]).T


        # Update the Addaptive Amplitudes for the next search
        # This should minimize the amplitude of ones that have large variance, and increase those which have high variance.
        Cost_Change_From_Probe_Samples  = Cost - PreviousCost

        Change_Vectors = All_Suggestions_From_Two_Mode_Section1 - np.array([Best_Individual]).T
        Gradients =   Cost_Change_From_Probe_Samples/(Change_Vectors + 0.00001)
        
        


        Update_Addaptive_Amplitudes(Gradients,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)

                 
        
        ## Update Directional Pools
        Update_Directional_Pool(Gradients,Cost_Change_From_Probe_Samples,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)

             


         ## Update Resource Allocation   
        Best_Cost_Change_From_Individuals1[:,i] = Update_Resource_Allocation(Cost_Change_From_Probe_Samples,Indexes_Of_The_Suggestions_From_The_Different_Algorithms1,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4,Hyperparameters_For_Symmetric_Conductor)
                                                                                                        
                                                                                                                                                                                                                                                                                                                                                        
        ## Update Best_Individuals_And_Costs_From_This_Round   
        Best_Individual = All_Suggestions_From_Two_Mode_Section1[:,index_min]

        Cost[index_min ] = 10000*Cost[index_min ]
        index_min = np.argmin(Cost)
        Second_Best_Individual = All_Suggestions_From_Two_Mode_Section1[:,index_min]
        PreviousCost = CurrentCost                                                                                                                
        Update_Best_Individuals_And_Costs_From_This_Round(Best_Individual,Second_Best_Individual,Parameter_ChangeVector,Cost_Change,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)                                                                                                                           
                                                                                                             
                                                                                                                                                                                                                                                             
        if print_Cost == 1:
            print("Run Summary:")
            print("---------------------------------------------------------------------")
            print("CurrentCost is:",CurrentCost,"After Running Iteration",current_iteration,"\\",Total_Number_Of_Iterations)
            print("the Random_Search_exponential_Growth_Factors are :",Suggestion_Tools_1.Random_Search_exponential_Growth_Factor,\
                  Suggestion_Tools_2.Random_Search_exponential_Growth_Factor,Suggestion_Tools_3.Random_Search_exponential_Growth_Factor,\
                  Suggestion_Tools_4.Random_Search_exponential_Growth_Factor)
            print("the Targeted_Search_exponential_Growth_Factor is:",Suggestion_Tools_1.Targeted_Search_exponential_Growth_Factor   ,\
                 Suggestion_Tools_2.Targeted_Search_exponential_Growth_Factor   ,Suggestion_Tools_3.Targeted_Search_exponential_Growth_Factor   ,\
                 Suggestion_Tools_4.Targeted_Search_exponential_Growth_Factor   )    
            print("---------------------------------------------------------------------")
            print("")            
            
          
    return Best_Individual,Second_Best_Individual,CurrentCost,PreviousCost,Best_Cost_Change_From_Individuals1,current_iteration


In [9]:

def Force_Suggested_Hyperparameters_To_be_Within_The_Allowed_Limits(Suggested_Hyper_Parameter_Samples_From_Two_Mode,Hyperparameter_Allowed_Values):

## The 0 second index is the lower bound of the allowed values, and the 1 second index is the upper bound

    Suggested_Hyper_Parameter_Samples_From_Two_Mode[0,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[0,:],Hyperparameter_Allowed_Values[0,0]),Hyperparameter_Allowed_Values[0,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[1,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[1,:],Hyperparameter_Allowed_Values[1,0]),Hyperparameter_Allowed_Values[1,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[2,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[2,:],Hyperparameter_Allowed_Values[2,0]),Hyperparameter_Allowed_Values[2,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[3,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[3,:],Hyperparameter_Allowed_Values[3,0]),Hyperparameter_Allowed_Values[3,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[4,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[4,:],Hyperparameter_Allowed_Values[4,0]),Hyperparameter_Allowed_Values[4,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[5,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[5,:],Hyperparameter_Allowed_Values[5,0]),Hyperparameter_Allowed_Values[5,1])
#     print("Before",Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:])
#     Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:] = np.minimum(np.maximum((10**Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:]),Hyperparameter_Allowed_Values[6,0]),Hyperparameter_Allowed_Values[6,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:],Hyperparameter_Allowed_Values[6,0]),Hyperparameter_Allowed_Values[6,1])
#     print("After",Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[7,:] = np.minimum(np.maximum(Suggested_Hyper_Parameter_Samples_From_Two_Mode[7,:],Hyperparameter_Allowed_Values[7,0]),Hyperparameter_Allowed_Values[7,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[8,:] = np.fmin(np.fmax(Suggested_Hyper_Parameter_Samples_From_Two_Mode[8,:],Hyperparameter_Allowed_Values[8,0]),Hyperparameter_Allowed_Values[8,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[9,:] = np.fmin(np.fmax(Suggested_Hyper_Parameter_Samples_From_Two_Mode[9,:],Hyperparameter_Allowed_Values[9,0]),Hyperparameter_Allowed_Values[9,1])
    Suggested_Hyper_Parameter_Samples_From_Two_Mode[10,:] = np.fmin(np.fmax(Suggested_Hyper_Parameter_Samples_From_Two_Mode[10,:],Hyperparameter_Allowed_Values[10,0]),Hyperparameter_Allowed_Values[10,1])

    
    Hyperparameters_1 = Suggested_Hyper_Parameter_Samples_From_Two_Mode[:,0]
    Hyperparameters_2 = Suggested_Hyper_Parameter_Samples_From_Two_Mode[:,1]
    Hyperparameters_3 = Suggested_Hyper_Parameter_Samples_From_Two_Mode[:,2]
    Hyperparameters_4 = Suggested_Hyper_Parameter_Samples_From_Two_Mode[:,3]

#     Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:] = np.log10(Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:])
#     print("After2",Suggested_Hyper_Parameter_Samples_From_Two_Mode[6,:])
    return Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggested_Hyper_Parameter_Samples_From_Two_Mode




    
# def Optimize_With_TSTM(InitialParameters,Cost1,NumberOfSamples,TargetedSearchDecayRate,RandomSearchGrowthRate,TargetedMultiplicationFactor ,RandomMultiplicationFactor,SignificantChangeValue,Maximal_Random_Search,print_Cost,Number_Of_Itterations,alpha,beta):
    
def Run_Optimization_With_TWO_Mode_Online_Hyperparameter_Addaptation(InitialParameters,fun,Total_Number_Of_Iterations,Number_Of_Iterations_With_Hyperparameters_Set,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4,Hyperparameters_For_Online_Hyperparameter_Search,print_Cost,Total_Resources,Hyperparameter_Allowed_Values):


    Number_Of_Iterations_With_These_Hyperparameters = Number_Of_Iterations_With_Hyperparameters_Set

    
    Random_Search_Growth_Rate  = Hyperparameters_For_Online_Hyperparameter_Search[0]
    Random_Search_Period = Hyperparameters_For_Online_Hyperparameter_Search[1]
    Maximal_Random_Search =Hyperparameters_For_Online_Hyperparameter_Search[2]
    Targeted_Search_Growth_Rate = Hyperparameters_For_Online_Hyperparameter_Search[3]
    Targeted_Search_Decay_Rate = Hyperparameters_For_Online_Hyperparameter_Search[4]
    Significant_Change_Value = Hyperparameters_For_Online_Hyperparameter_Search[5]
    alpha = Hyperparameters_For_Online_Hyperparameter_Search[6]
    beta = Hyperparameters_For_Online_Hyperparameter_Search[7]
    Hyper_Parameter_Directional_Pool_Ratio = Hyperparameters_For_Online_Hyperparameter_Search[8]
    Hyper_Parameter_Pool_Size = Hyperparameters_For_Online_Hyperparameter_Search[9]
    Hyper_Parameter_Pool_Beta = Hyperparameters_For_Online_Hyperparameter_Search[10]
    
    Hyper_Parameter_Directional_Pool = np.random.rand(11,int(Hyper_Parameter_Pool_Size))
    ## Maybe add the three hyperparameters for the directional pool here too
    
    
    ## Initialize the suggestion tools for the hyperparameter optimization
    Random_Search_exponential_Growth_Factor = 0
    Targeted_Search_exponential_Growth_Factor = 0
    Hyper_Parameter_ChangeVector = np.zeros(len(Hyperparameters_1))
    Hyper_Parameter_Cost_Change = 0
    current_iteration = 0
    
    
    ## This is to set the hyperparameters for optimization
    Number_Of_Samples = Number_Of_Algorithms
    Best_Individual_Hyperparameters = Hyperparameters_1
    Second_Best_Individual_Hyperparameters = Hyperparameters_1
    
    ## This is to set the function parameters for optimization    
    Best_Individual_Parameters =     InitialParameters
    Second_Best_Individual_Parameters =     InitialParameters
    PreviousCost = fun(InitialParameters)


    ## Update Directional Pools with random gradients
    Random_Parameter_Gradients = np.random.rand(len(InitialParameters),Total_Resources)
    Random_Parameter_Gradients[:,0] = 0
    Cost_Change_From_Probe_Samples = -np.ones(Total_Resources)
    Update_Directional_Pool(Random_Parameter_Gradients,Cost_Change_From_Probe_Samples,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)

    ## This is find the Cost of the first set of hyperparameters
    Best_Individual_Parameters,Second_Best_Individual_Parameters,CurrentCost,PreviousCost,Best_Cost_Change_From_Individuals1,current_iteration =  Optimize_With_TWO_Mode_Section(Best_Individual_Parameters,Second_Best_Individual_Parameters,PreviousCost,fun,current_iteration,Total_Number_Of_Iterations,Number_Of_Iterations_With_These_Hyperparameters,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)

    Cost_Function_For_Hyperparameter_Optimization =( np.mean(Best_Cost_Change_From_Individuals1,1) + np.max(Best_Cost_Change_From_Individuals1,1))/2
    
    index_min = np.argmin(Cost_Function_For_Hyperparameter_Optimization)
    Current_Hyperparameter_Cost = Cost_Function_For_Hyperparameter_Optimization[index_min ]

    Previous_Hyperparameter_Cost = Cost_Function_For_Hyperparameter_Optimization[index_min]


    Adaptive_Amplitude = np.ones((len(Hyperparameters_1),Number_Of_Samples))
    Cost_Function_For_Hyperparameter_Optimization = np.zeros(Number_Of_Samples)
    S = 0
    
       
    Number_Of_Hyperparameter_Iterations = np.floor(Total_Number_Of_Iterations/Number_Of_Iterations_With_These_Hyperparameters)

    for i in range(0,int(Number_Of_Hyperparameter_Iterations)):
        

        ## Generate suggested hyperparameters    
        Suggested_Hyper_Parameter_Samples_From_Two_Mode, Random_Search_exponential_Growth_Factor, Targeted_Search_exponential_Growth_Factor =     Two_Mode_Optimizer(Best_Individual_Hyperparameters,Second_Best_Individual_Hyperparameters,Number_Of_Samples,Hyper_Parameter_ChangeVector,Hyper_Parameter_Cost_Change, Random_Search_exponential_Growth_Factor,Targeted_Search_exponential_Growth_Factor,Targeted_Search_Decay_Rate,Random_Search_Growth_Rate,Targeted_Search_Growth_Rate,Random_Search_Period,Significant_Change_Value,Maximal_Random_Search,Adaptive_Amplitude,Hyper_Parameter_Directional_Pool_Ratio, Hyper_Parameter_Pool_Size, Hyper_Parameter_Directional_Pool)
        ## Force the hyperparameters to be within the allowed range

        Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggested_Hyper_Parameter_Samples_From_Two_Mode = Force_Suggested_Hyperparameters_To_be_Within_The_Allowed_Limits(Suggested_Hyper_Parameter_Samples_From_Two_Mode,Hyperparameter_Allowed_Values)
 

        ## Update Directional Pools with random gradients
        Random_Parameter_Gradients = np.random.rand(len(InitialParameters),Total_Resources)
        Random_Parameter_Gradients[:,0] = 0
        Cost_Change_From_Probe_Samples = -np.ones(Total_Resources)
        Re_Initialize_Directional_Pool(Random_Parameter_Gradients,Cost_Change_From_Probe_Samples,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)

    
    
        ## Try to optimize with these hyperparameters (Maybe output the suggestion tools as well?)
        Best_Individual_Parameters,Second_Best_Individual_Parameters,CurrentCost,PreviousCost,Best_Cost_Change_From_Individuals1,current_iteration =  Optimize_With_TWO_Mode_Section(Best_Individual_Parameters,Second_Best_Individual_Parameters,PreviousCost,fun,current_iteration,Total_Number_Of_Iterations,Number_Of_Iterations_With_These_Hyperparameters,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4)
       ## Calculate the Cost_Function_For_Hyperparameter_Optimization
    

        Cost_Function_For_Hyperparameter_Optimization =( np.mean(Best_Cost_Change_From_Individuals1,1) + np.min(Best_Cost_Change_From_Individuals1,1))/2
        Cost_Function_For_Hyperparameter_Optimization1 = np.array([Cost_Function_For_Hyperparameter_Optimization[0],Cost_Function_For_Hyperparameter_Optimization[1],Cost_Function_For_Hyperparameter_Optimization[2],Cost_Function_For_Hyperparameter_Optimization[3]])

        
        ## Find the Best set of hyperparameters and the change vectors
        index_min = np.argmin(Cost_Function_For_Hyperparameter_Optimization)
        Current_Hyperparameter_Cost = Cost_Function_For_Hyperparameter_Optimization[index_min ]
        Hyper_Parameter_Cost_Change = Current_Hyperparameter_Cost - Previous_Hyperparameter_Cost
        Hyper_Parameter_ChangeVector = Suggested_Hyper_Parameter_Samples_From_Two_Mode[:,index_min] - Best_Individual_Hyperparameters


        
        ## This is the RMS Amplitude Addaptation - For the hyperparameter search
        Cost_Change_From_Probe_Samples  = Cost_Function_For_Hyperparameter_Optimization - Cost_Function_For_Hyperparameter_Optimization[0]

        Change_Vectors = Suggested_Hyper_Parameter_Samples_From_Two_Mode - np.array([Best_Individual_Hyperparameters]).T
        Gradient =   Cost_Change_From_Probe_Samples/(Change_Vectors + 0.001)

        S = beta*S + (1-beta)*(np.mean(np.abs(Gradient),1))**2
        S = S*np.sign(np.mean(Gradient,1))
        Adaptive_Amplitude1 = alpha/np.sqrt(np.fmax(S + alpha**2,0.1*alpha**2))
        Adaptive_Amplitude[:,:] = np.array([Adaptive_Amplitude1]).T ## this makes an parameter wise adaptive amplitude Based on RMS prob, which we can do element wise multiplication with the random searches.
        # This should minimize the amplitude of ones that have large variance, and increase those which have high variance.

        
        ## Save the Best Sets of hyperparameters and Cost Function
        Best_Individual_Hyperparameters = Suggested_Hyper_Parameter_Samples_From_Two_Mode[:,index_min]
        Cost_Function_For_Hyperparameter_Optimization[index_min ] = (10**10) + Cost_Function_For_Hyperparameter_Optimization[index_min ]
        index_min = np.argmin(Cost_Function_For_Hyperparameter_Optimization)
        Second_Best_Individual_Hyperparameters = Suggested_Hyper_Parameter_Samples_From_Two_Mode[:,index_min]
        

        Previous_Hyperparameter_Cost = Current_Hyperparameter_Cost
        

        
        if print_Cost == 1:
            print("hyperparameter Summary:")
            print("---------------------------------------------------------------------")
#             print("CurrentCost is:",CurrentCost,"After Running Iteration",current_iteration,"\\",Total_Number_Of_Iterations)
#             print("")
            print("After Running hyperparameter Iteration",i,"\\",int(Number_Of_Hyperparameter_Iterations))
            print("Cost_Function_For_Hyperparameter_Optimization",Cost_Function_For_Hyperparameter_Optimization1)
           
            print("")
            print("Best_Individual_Hyperparameters",Best_Individual_Hyperparameters)
            print("")
            print("the Random_Search_exponential_Growth_Factor for the hyperparameter search is:",Random_Search_exponential_Growth_Factor)
            print("the Targeted_Search_exponential_Growth_Factor for the hyperparameter search is:",Targeted_Search_exponential_Growth_Factor)    
    
            print("---------------------------------------------------------------------")
            print("")            
            

    return Best_Individual_Parameters


In [10]:
def Set_Hyperparameter_Ranges(Random_Search_Growth_Rate_Range, Random_Search_Period_Range   ,Maximal_Random_Search_Range,Targeted_Search_Growth_Rate_Range ,Targeted_Search_Decay_Rate_Range ,Significant_Change_Value_Range,alpha_Range ,beta_Range ,Directional_Pool_Ratio_Range,Pool_Size_Range,Pool_beta_Range):
    Hyperparameter_Allowed_Values = np.zeros((11,2))

    Hyperparameter_Allowed_Values[0,:] = Random_Search_Growth_Rate_Range
    Hyperparameter_Allowed_Values[1,:] = Random_Search_Period_Range    
    Hyperparameter_Allowed_Values[2,:] = Maximal_Random_Search_Range    
    Hyperparameter_Allowed_Values[3,:] = Targeted_Search_Growth_Rate_Range    
    Hyperparameter_Allowed_Values[4,:] = Targeted_Search_Decay_Rate_Range   
    Hyperparameter_Allowed_Values[5,:] = Significant_Change_Value_Range
    Hyperparameter_Allowed_Values[6,:] = alpha_Range
    Hyperparameter_Allowed_Values[7,:] = beta_Range
    Hyperparameter_Allowed_Values[8,:] = Directional_Pool_Ratio_Range
    Hyperparameter_Allowed_Values[9,:] = Pool_Size_Range
    Hyperparameter_Allowed_Values[10,:] = Pool_beta_Range

    return Hyperparameter_Allowed_Values


def Mean_ChangeValue_fit(Maximal_Random_Search,Number_Of_Parameters):
    fit_Parameters = [-0.007921662,0.003755446,5.92*(10**(-5)),-2.45*(10**(-7)),0.250014494] # These were found numerically from the file "Finding_The_Segnificant_Change_Value.ipynb"
    return    fit_Parameters[0]*Maximal_Random_Search + fit_Parameters[1]*Maximal_Random_Search**2 + fit_Parameters[2]*Number_Of_Parameters + fit_Parameters[3]*Number_Of_Parameters**2 + fit_Parameters[4]*Maximal_Random_Search*Number_Of_Parameters
    




In [11]:
def Orchestra_Optimize_With_Pinball_Section(x0,fun,Number_Of_Iterations,print_Cost):

    ## This will be the Two Mode Conductor

    Number_Of_Parameters = len(x0)
    InitialParameters = x0
    Total_Number_Of_Iterations = Number_Of_Iterations
    
    Number_Of_Iterations_With_Hyperparameters_Set = 5
    
    

    ############################## Setting up The Conductor (Resource allocation) algorithm ###################################
    Total_Resources = 60
    Number_Of_Algorithms = 4
    Mass =10
    Self_Spring_Constants = 10
    Interaction_Spring_Constants = 40

    Hyperparameters_For_Symmetric_Conductor = Set_Hyperparameters_For_Symmetric_Conductor(Total_Resources,Number_Of_Algorithms,Mass,Self_Spring_Constants,Interaction_Spring_Constants)
    Initial_Resource_Allocation = Hyperparameters_For_Symmetric_Conductor[:,0]
    Current_Resource_Allocation = Hyperparameters_For_Symmetric_Conductor[:,0]

    Changevector = np.array([0, 0 ,0 ,0]) ## Negative is bad
    Recomended_Resource_Allocation = Symmetric_Section_Conductor(Current_Resource_Allocation,Changevector,Hyperparameters_For_Symmetric_Conductor) 
    Current_Resource_Allocation = Recomended_Resource_Allocation
#     print("Current_Resource_Allocation",Current_Resource_Allocation)
    #############################################################################################################################




    
    ######################### Setting the hyperparameter ranges ######################################################################
    Random_Search_Growth_Rate_Range   = np.array([0.01,10])
    Random_Search_Period_Range  = np.array([0.1,100])
    Maximal_Random_Search_Range  = np.array([0.00001,10])
    Targeted_Search_Growth_Rate_Range  = np.array([0.01,5])
    Targeted_Search_Decay_Rate_Range  = np.array([0.1,100]) 
    Significant_Change_Value_Range  = np.array([10**-30,0.0001]) 
    alpha_Range = np.array([10**-5,10**15]) ## (Here Do 10**random)
    beta_Range = np.array([0.1,1]) 
    Directional_Pool_Ratio_Range = np.array([0.0001,1]) 
    Pool_Size_Range = np.array([0.1,2.0]) 
    Pool_beta_Range = np.array([0.1,1]) 

    Hyperparameter_Allowed_Values = Set_Hyperparameter_Ranges(Random_Search_Growth_Rate_Range, Random_Search_Period_Range   ,Maximal_Random_Search_Range,Targeted_Search_Growth_Rate_Range ,Targeted_Search_Decay_Rate_Range ,Significant_Change_Value_Range,alpha_Range ,beta_Range ,Directional_Pool_Ratio_Range,Pool_Size_Range,Pool_beta_Range)
    #############################################################################################################################



    ############################### Setting the Hyperparameters_For_Online_Hyperparameter_improvement  Algorithm #########################
    Hyperparameters_For_Online_Hyperparameter_Search = np.zeros(11)
    Hyperparameters_For_Online_Hyperparameter_Search[0] =5.0   # Random_Search_Growth_Rate 
    Hyperparameters_For_Online_Hyperparameter_Search[1] =5.0    # Random_Search_Period 
    Hyperparameters_For_Online_Hyperparameter_Search[2] =20.02   # Maximal_Random_Search 
    Hyperparameters_For_Online_Hyperparameter_Search[3] =0.5    # Targeted_Search_Growth_Rate
    Hyperparameters_For_Online_Hyperparameter_Search[4] =2.0    # Targeted_Search_Decay_Rate 
    Hyperparameters_For_Online_Hyperparameter_Search[5] =0.00001*Mean_ChangeValue_fit(Hyperparameters_For_Online_Hyperparameter_Search[2] ,8)    # Significant_Change_Value
    Hyperparameters_For_Online_Hyperparameter_Search[6] =(10**4)/Hyperparameters_For_Online_Hyperparameter_Search[2]    # alpha 
    Hyperparameters_For_Online_Hyperparameter_Search[7] =0.6     # beta
    Hyperparameters_For_Online_Hyperparameter_Search[8] =0.6     # directional ratio
    Hyperparameters_For_Online_Hyperparameter_Search[9] =3     # pool size
    Hyperparameters_For_Online_Hyperparameter_Search[10] =0.7     # pool beta
    #############################################################################################################################


    
    
    
    
    ########################initialize the different Suggestion_Tools For the Two Mode and the hyperparameters #######################
    Suggestion_Tools_1 = Create_New_Two_Mode_Suggestion_Tools_Object(Number_Of_Parameters)
    Suggestion_Tools_2 = Create_New_Two_Mode_Suggestion_Tools_Object(Number_Of_Parameters)
    Suggestion_Tools_3 = Create_New_Two_Mode_Suggestion_Tools_Object(Number_Of_Parameters)
    Suggestion_Tools_4 = Create_New_Two_Mode_Suggestion_Tools_Object(Number_Of_Parameters)

    Suggestion_Tools_1.Number_Of_Samples =      Current_Resource_Allocation[0]                                                                                                                     
    Suggestion_Tools_2.Number_Of_Samples =      Current_Resource_Allocation[1] 
    Suggestion_Tools_3.Number_Of_Samples =      Current_Resource_Allocation[2]                                                                                                                                  
    Suggestion_Tools_4.Number_Of_Samples =      Current_Resource_Allocation[3]  



    ## This is to initiailize the Hyperparameters - Note that they should all be positive numbers (Maybe make 6 and 7 be 10**rand(2))
    Initial_Hyper_Parameter_Samples = 10*np.random.rand(11,Number_Of_Algorithms)
    Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Initial_Hyper_Parameter_Samples = Force_Suggested_Hyperparameters_To_be_Within_The_Allowed_Limits(Initial_Hyper_Parameter_Samples,Hyperparameter_Allowed_Values)
    #############################################################################################################################



    Best_Parameters = Run_Optimization_With_TWO_Mode_Online_Hyperparameter_Addaptation(InitialParameters,fun,Total_Number_Of_Iterations,Number_Of_Iterations_With_Hyperparameters_Set,Hyperparameters_1,Hyperparameters_2,Hyperparameters_3,Hyperparameters_4,Suggestion_Tools_1,Suggestion_Tools_2,Suggestion_Tools_3,Suggestion_Tools_4,Hyperparameters_For_Online_Hyperparameter_Search,print_Cost,Total_Resources,Hyperparameter_Allowed_Values)

    return Best_Parameters

In [18]:
def fun(x): ## Ellispod
#     return np.sum((x-2)**2) + np.sum(np.sin(x))

    a = 1000000**(np.arange(D)/D)
    return np.sum(a*((x-X_Optimal)**2) )

Number_Of_Iterations = 2000

D = 10
x0 = 10*(np.random.rand(D)-0.5)
X_Optimal = 10*(np.random.rand(D)-0.5)

print_Cost = 0
Print_Two_Mode_Analytics = 0

print("Initial Cost",fun(x0))
Best_Parameters_After_Optimization = Orchestra_Optimize_With_Two_Mode_Section(x0,fun,Number_Of_Iterations,print_Cost)
print("Final Cost",fun(Best_Parameters_After_Optimization))
print("log10 Total_Evaluations /D= ",np.log10(60*Number_Of_Iterations/D))


Initial Cost 4028914.409437531


C:\Users\benjy\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: overflow encountered in square


Final Cost 2.65380651641211
log10 Total_Evaluations /D=  4.079181246047625


In [13]:

## Defining Cost Function
class Two_Mode_Section:
    
    Name = 'Two_Mode_Section'

    def __init__(self, Name):
        self.__name__ = 'Two_Mode_Section'

        
    def Optimize(fun, x0, Number_Of_Iterations ): 
        print_Cost = 0

        if len(x0) >= 5:
            print_Cost = 0

        Best_Parameters_After_Optimization = Orchestra_Optimize_With_Two_Mode_Section(x0,fun,Number_Of_Iterations,print_Cost)
        
  
        x_min = Best_Parameters_After_Optimization
        return x_min
    
    
    
    
    
SOLVER = Two_Mode_Section

print(SOLVER.__name__)
print(SOLVER.__name__ == 'Two_Mode_Section')

def fun(x):
    return np.sum((x-2)**2)


x0= np.random.rand(20)
remaining_evals = 500
print(fun(x0))
x_min1  = SOLVER.Optimize(fun, x0,remaining_evals)
print(fun(x_min1))

Two_Mode_Section
True
47.12590898523435
2.92033535691153e-12


C:\Users\benjy\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: overflow encountered in square


In [13]:
#!/usr/bin/env python
"""Python script for the COCO experimentation module `cocoex`.

Usage from a system shell::

    python example_experiment.py bbob

runs a full but short experiment on the bbob suite. The optimization
algorithm used is determined by the `SOLVER` attribute in this file::

    python example_experiment.py bbob 20

runs the same experiment but with a budget of 20 * dimension
f-evaluations::

    python example_experiment.py bbob-biobj 1e3 1 20

runs the first of 20 batches with maximal budget of
1000 * dimension f-evaluations on the bbob-biobj suite.
All batches must be run to generate a complete data set.

Usage from a python shell:

>>> import example_experiment as ee
>>> ee.suite_name = "bbob-biobj"
>>> ee.SOLVER = ee.random_search  # which is default anyway
>>> ee.observer_options['algorithm_info'] = '"default of example_experiment.py"'
>>> ee.main(5, 1+9, 2, 300)  # doctest: +ELLIPSIS
Benchmarking solver...

runs the 2nd of 300 batches with budget 5 * dimension and at most 9 restarts.

Calling `example_experiment` without parameters prints this
help and the available suite names.
"""
from __future__ import absolute_import, division, print_function, unicode_literals
try: range = xrange
except NameError: pass
import os, sys
import time
import numpy as np  # "pip install numpy" installs numpy
import cocoex
from scipy import optimize # for tests with fmin_cobyla
from cocoex import Suite, Observer, log_level
del absolute_import, division, print_function, unicode_literals

verbose = 1

try: import cma  # cma.fmin is a solver option, "pip install cma" installs cma
except: pass
try: from scipy.optimize import fmin_slsqp  # "pip install scipy" installs scipy
except: pass
try: range = xrange  # let range always be an iterator
except NameError: pass

from cocoex import default_observers  # see cocoex.__init__.py
from cocoex.utilities import ObserverOptions, ShortInfo, ascetime, print_flush
from cocoex.solvers import random_search

def default_observer_options(budget_=None, suite_name_=None, current_batch_=None):
    """return defaults computed from input parameters or current global vars
    """
    global budget, suite_name, number_of_batches, current_batch
    if budget_ is None:
        budget_ = budget
    if suite_name_ is None:
        suite_name_ = suite_name
    if current_batch_ is None and number_of_batches > 1:
        current_batch_ = current_batch
    opts = {}
    try:
        opts.update({'result_folder': '"%s_on_%s%s_budget%04dxD"'
                    % (SOLVER.__name__,
                       suite_name_,
                       "" if current_batch_ is None
                          else "_batch%03dof%d" % (current_batch_, number_of_batches),
                       budget_)})
    except: pass
    try:
        solver_module = '(%s)' % SOLVER.__module__
    except:
        solver_module = ''
    try:
        opts.update({'algorithm_name': SOLVER.__name__ + solver_module})
    except: pass
    return opts

# ===============================================
# loops over a benchmark problem suite
# ===============================================
def batch_loop(solver, suite, observer, budget,
               max_runs, current_batch, number_of_batches):
    """loop over all problems in `suite` calling
    `coco_optimize(solver, problem, budget * problem.dimension, max_runs)`
    for each eligible problem.

    A problem is eligible if ``problem_index + current_batch - 1``
    modulo ``number_of_batches`` equals ``0``.

    This distribution into batches is likely to lead to similar
    runtimes for the batches, which is usually desirable.
    """
    addressed_problems = []
    short_info = ShortInfo()
    for problem_index, problem in enumerate(suite):
        if (problem_index + current_batch - 1) % number_of_batches:
            continue
        observer.observe(problem)
        short_info.print(problem) if verbose else None
        runs = coco_optimize(solver, problem, budget * problem.dimension,
                             max_runs)
        if verbose:
            print_flush("!" if runs > 2 else ":" if runs > 1 else ".")
        short_info.add_evals(problem.evaluations + problem.evaluations_constraints, runs)
        problem.free()  # not necessary as `enumerate` tears the problem down
        addressed_problems += [problem.id]
    print(short_info.function_done() + short_info.dimension_done())
    short_info.print_timings()
    print("  %s done (%d of %d problems benchmarked%s)" %
           (suite_name, len(addressed_problems), len(suite),
             ((" in batch %d of %d" % (current_batch, number_of_batches))
               if number_of_batches > 1 else "")), end="")
    if number_of_batches > 1:
        print("\n    MAKE SURE TO RUN ALL BATCHES", end="")
    return addressed_problems

#===============================================
# interface: ADD AN OPTIMIZER BELOW
#===============================================
def coco_optimize(solver, fun, max_evals, max_runs=1e9):
    """`fun` is a callable, to be optimized by `solver`.

    The `solver` is called repeatedly with different initial solutions
    until either the `max_evals` are exhausted or `max_run` solver calls
    have been made or the `solver` has not called `fun` even once
    in the last run.

    Return number of (almost) independent runs.
    """
    range_ = fun.upper_bounds - fun.lower_bounds
    center = fun.lower_bounds + range_ / 2
    if fun.evaluations:
        print('WARNING: %d evaluations were done before the first solver call' %
              fun.evaluations)

    for restarts in range(int(max_runs)):
        remaining_evals = max_evals - fun.evaluations - fun.evaluations_constraints
        x0 = center + (restarts > 0) * 0.8 * range_ * (
                np.random.rand(fun.dimension) - 0.5)
        fun(x0)  # can be incommented, if this is done by the solver

        if solver.__name__ in ("random_search", ):
            solver(fun, fun.lower_bounds, fun.upper_bounds,
                   remaining_evals)
        elif solver.__name__ == 'fmin' and solver.__globals__['__name__'] in ['cma', 'cma.evolution_strategy', 'cma.es']:
            if x0[0] == center[0]:
                sigma0 = 0.02
                restarts_ = 0
            else:
                x0 = "%f + %f * np.random.rand(%d)" % (
                        center[0], 0.8 * range_[0], fun.dimension)
                sigma0 = 0.2
                restarts_ = 6 * (observer_options.as_string.find('IPOP') >= 0)

            solver(fun, x0, sigma0 * range_[0], restarts=restarts_,
                   options=dict(scaling=range_/range_[0], maxfevals=remaining_evals,
                                termination_callback=lambda es: fun.final_target_hit,
                                verb_log=0, verb_disp=0, verbose=-9))
        elif solver.__name__ == 'fmin_slsqp':
            solver(fun, x0, iter=1 + remaining_evals / fun.dimension,
                   iprint=-1)
        elif solver.__name__ in ("fmin_cobyla", ):
            x0 = fun.initial_solution
            solver(fun, x0, lambda x: -fun.constraint(x), maxfun=remaining_evals,
                   disp=0, rhoend=1e-9)
############################ ADD HERE ########################################
        # ### IMPLEMENT HERE THE CALL TO ANOTHER SOLVER/OPTIMIZER ###
        elif solver.__name__ == 'Two_Mode_Section':
        #     CALL MY SOLVER, interfaces vary

            solver.Optimize(fun, x0,remaining_evals)

##############################################################################
        else:
            solver(fun, x0)

        if fun.evaluations + fun.evaluations_constraints >= max_evals or \
           fun.final_target_hit:
            break
        # quit if fun.evaluations did not increase
        still_remaining = max_evals - fun.evaluations - fun.evaluations_constraints
        if still_remaining >= remaining_evals:  # break loop if no evaluations were done
            if still_remaining > remaining_evals:
                raise RuntimeError("function evaluations decreased")
            if still_remaining >= fun.dimension + 2:
                print("WARNING: %d evaluations of budget %d remaining" %
                      (still_remaining, max_evals))
            break
    return 1 + restarts  # number of (almost) independent launches of `solver`

# ===============================================
# set up: CHANGE HERE SOLVER AND FURTHER SETTINGS AS DESIRED
# ===============================================
######################### CHANGE HERE ########################################
# CAVEAT: this might be modified from input args
suite_name = "bbob"  # always overwritten when called from system shell
                     # see available choices via cocoex.known_suite_names
budget = 100  # maxfevals = budget x dimension ### INCREASE budget WHEN THE DATA CHAIN IS STABLE ###
max_runs = 1e9  # number of (almost) independent trials per problem instance
number_of_batches = 1  # allows to run everything in several batches
current_batch = 1      # 1..number_of_batches
##############################################################################
# By default we call SOLVER(fun, x0), but the INTERFACE CAN BE ADAPTED TO EACH SOLVER ABOVE

# SOLVER = random_search
# SOLVER = Basic_Two_Mode_Object_first_draft

# SOLVER = fmin_slsqp
SOLVER = Two_Mode_Section

# SOLVER = optimize.fmin_cobyla
# SOLVER = my_solver # SOLVER = fmin_slsqp # SOLVER = cma.fmin
suite_instance = "" # "year:2016"
suite_options = ""  # "dimensions: 2,3,5,10,20 "  # if 40 is not desired
# for more suite options, see http://numbbo.github.io/coco-doc/C/#suite-parameters
observer_options = ObserverOptions({  # is (inherited from) a dictionary
                    'algorithm_info': '"A SIMPLE RANDOM SEARCH ALGORITHM"', # CHANGE/INCOMMENT THIS!
                    # 'algorithm_name': '',  # default already provided from SOLVER name
                    # 'result_folder': '',  # default already provided from several global vars
                   })
######################### END CHANGE HERE ####################################

# ===============================================
# run (main)
# ===============================================
def main(budget=budget,
         max_runs=max_runs,
         current_batch=current_batch,
         number_of_batches=number_of_batches):
    """Initialize suite and observer, then benchmark solver by calling
    ``batch_loop(SOLVER, suite, observer, budget,...``
    """
    suite = Suite(suite_name, suite_instance, suite_options)

    observer_name = default_observers()[suite_name]
    # observer_name = another observer if so desired
    observer_options.update_gracefully(default_observer_options())
    observer = Observer(observer_name, observer_options.as_string)

    print("Benchmarking solver '%s' with budget=%d*dimension on %s suite, %s"
          % (' '.join(str(SOLVER).split()[:2]), budget,
             suite.name, time.asctime()))
    if number_of_batches > 1:
        print('Batch usecase, make sure you run *all* %d batches.\n' %
              number_of_batches)
    t0 = time.clock()
    batch_loop(SOLVER, suite, observer, budget, max_runs,
               current_batch, number_of_batches)
    print(", %s (%s total elapsed time)." %
            (time.asctime(), ascetime(time.clock() - t0)))
    print('Data written to folder', observer.result_folder)
    print('To post-process the data call \n'
          '    python -m cocopp %s \n'
          'from a system shell or \n'
          '    cocopp.main("%s") \n'
          'from a python shell' % (2 * (observer.result_folder,)))

# ===============================================
if __name__ == '__main__':
    """read input parameters and call `main()`"""
#     print(sys.argv)
    sys.argv[1] = 'bbob'
    sys.argv = sys.argv[0:2]
#     print(sys.argv)
    if len(sys.argv) < 2 or sys.argv[1] in ["--help", "-h"]:
            print(__doc__)
            print("Recognized suite names: " + str(cocoex.known_suite_names))
            sys.exit(0)
    suite_name = sys.argv[1]
    suite_name = 'bbob'
    if suite_name not in cocoex.known_suite_names:
        print('WARNING: "%s" not in known names %s' %
                (suite_name, str(cocoex.known_suite_names)))
    if len(sys.argv) > 2:
        budget = float(sys.argv[2])
    if len(sys.argv) > 3:
        current_batch = int(sys.argv[3])
    if len(sys.argv) > 4:
        number_of_batches = int(sys.argv[4])
    if len(sys.argv) > 5:
        messages = ['Argument "%s" disregarded (only 4 arguments are recognized).' % sys.argv[i]
            for i in range(5, len(sys.argv))]
        messages.append('See "python example_experiment.py -h" for help.')
        raise ValueError('\n'.join(messages))
    main(budget, max_runs, current_batch, number_of_batches)


Benchmarking solver '<class '__main__.Two_Mode_Section'>' with budget=1000*dimension on b'bbob' suite, Sat Feb 23 11:12:08 2019
Feb23 11h12:08, d=2, running: f001......

KeyboardInterrupt: 

In [ ]:

# a = float('NaN')
# b = 3
# if np.isnan(b):
#     print(1)

In [ ]:
## With the two mode section I ran a 60 eval*D test - which ran into occasional errors in the f19, f6 - 
## All in all this test took 1:09 hours and 9 minutes